## Instrument Sandbox

Each cell contains example usages of all the instruments

Very useful for sending single commands, like using the CryoSwitch to change fridge lines, or using the SG_Anritsu to turn the TWPA on/off


In [ ]:
%load_ext autoreload
%autoreload 3

import sys, pyvisa, time
from pathlib import Path

current_path = Path(".")
misc_path = current_path / "misc"
instruments_path = current_path / "instruments"

# easy access to all instrument drivers
sys.path.append(str(current_path))
sys.path.append(str(misc_path))
sys.path.append(str(instruments_path))

pyvisa.log_to_screen(False)

### Cryoswitch

In [ ]:
from misc.CryoSwitchController.CryoSwitchController import Cryoswitch
import time 

if "switch" not in locals():
    switch = Cryoswitch(IP="192.168.0.117")
    switch.start() ## -> Initialization of the internal hardware
# switch.get_pulse_history(pulse_number=3, port='B')      ## -> Show the last 5 pulses send through on port A

# switch.get_internal_temperature()
# switch.set_output_voltage(5)        ## -> Set the output pulse voltage to 5V

contacts_disconnect = [3]
contacts_connect = [1]

sleep_time = 0.5  # seconds
    
for disconnect in contacts_disconnect:
    print(f"Disconnecting switch A - contact {disconnect}")
    profile = switch.disconnect(port='A', contact=disconnect)
    time.sleep(sleep_time)
    profile = switch.disconnect(port='B', contact=disconnect)
    time.sleep(sleep_time)


for connect in contacts_connect:
    print(f"Connecting switch B - contact {connect}")
    profile = switch.connect(port='A', contact=connect)
    time.sleep(sleep_time)
    profile = switch.connect(port='B', contact=connect)
    time.sleep(sleep_time)
    



### Minicircuits RF Switch

In [ ]:
misc_path = current_path / "misc" / "MiniCircuits"
sys.path.append(str(misc_path))

from MC_RFSwitch import MC_RFSwitch
ip_addr = "192.168.0.115"  

RF_Switch = MC_RFSwitch(ip_addr, debug=True)

# switches A, B, C, D go clockwise around the face of the switch, starting top left

# setting a switch to "1" 
#   - physically the port on the left labeled GREEN
#   - "1" in the GUI, also BLUE

# setting a switch to "2" 
#   - physically the port on the right labeled RED
#   - "2" in the GUI, also RED

    
"""
for cooldown 59, as of 1/23:

    switches A & B are for the cryoswitch line
        Switch A for fridge input (port 24)
            A1 = VNA output
            A2 = QICK output
        Switch B for fridge output (port 27)
            B1 - VNA input
            B2 - QICK input
            
    switches C & D are for the 3D qubit line
        Switch C for qubit line output (port 25)      
            C1 - Empty
            C2 - Empty   
        Switch D for qubit line input (port 23)
            D1 - Empty
            D2 - Empty

"""

RF_Switch.Set_Switch_State("A", 2)  # A = line 24  (input)
RF_Switch.Set_Switch_State("B", 2)  # B = line 27  (output)
RF_Switch.Set_Switch_State("C", 2)  # C = line 25  (output)
RF_Switch.Set_Switch_State("D", 2)  # D = line 23  (input)


# line_24_27 = 1    
# line_23_25 = 1

# RF_Switch.Set_Switch_State("A",line_24_27)  # A = line 24  (input)
# RF_Switch.Set_Switch_State("B",line_24_}27)  # B = line 27  (output)
# RF_Switch.Set_Switch_State("C",line_23_25)  # C = line 25  (output)
# RF_Switch.Set_Switch_State("D",line_23_25)  # D = line 23  (input)



### MC_VarAttenuator

In [ ]:
%load_ext autoreload
%autoreload 3

import sys, pyvisa, time
from pathlib import Path

current_path = Path(".")
misc_path = current_path / "misc" / "MiniCircuits"
instruments_path = current_path / "instruments"

# easy access to all instrument drivers
sys.path.append(str(current_path))
sys.path.append(str(misc_path))         
sys.path.append(str(instruments_path))

pyvisa.log_to_screen(False)

from MC_VarAttenuator import MC_VarAttenuator
ip_addr_1 = "192.168.0.113"  # atten #1
ip_addr_2 = "192.168.0.115"  # atten #2
    
atten_1 = MC_VarAttenuator(ip_addr_1)
atten_1.Set_Attenuation(0)
    
atten_2 = MC_VarAttenuator(ip_addr_2)
atten_2.Set_Attenuation(0)

pass

### Anritsu Signal Generator


In [ ]:
from SG_Anritsu import SG_Anritsu

TEST_Config = {
    "instrument_name" : "TEST",
    "rm_backend" : None,
    "instr_address" : 'GPIB::7::INSTR',  # test instr
}

TWPA_Config = {
    "instrument_name" : "TWPA",
    "rm_backend" : None,
    "instr_address" : 'GPIB::8::INSTR',  # twpa
}

pyvisa.log_to_screen(False)

if "TEST_SG" not in locals():
    TEST_SG = SG_Anritsu(TEST_Config, debug=True)

if "TWPA_SG" not in locals():
    TWPA_SG = SG_Anritsu(TWPA_Config, debug=True)

All_SGs = [TWPA_SG]

for SG in All_SGs:  
    # SG.open_pyvisa_backend()
    # SG.open_pyvisa_resource()
    # SG.debug_force_clear()
    # print(SG.resource.write("*CLS"))
    # print(SG.resource.query("*IDN?"))
     
    # # time.sleep(2)
    print(f"{SG.resource.query("OUTP:STAT?") = }")
    
    # print(f"{SG.get_power() = }")
    print(f"{SG.set_power(-17) = }")
    
    # print(f"{SG.get_freq() = }")
    print(f"{SG.set_freq(7.909e9) = }")
    
    # print(f"{SG.get_output() = }")
    print(f"{SG.set_output(True) = }")
    
    # SG.set_output(True)
    
    
    
    




### Agilent VNA


In [ ]:
from VNA_Keysight import VNA_Keysight

VNA_Keysight_InstrConfig = {
    "instrument_name" : "VNA_Keysight",
    # "rm_backend" : "@py",+
    "rm_backend" : None,
    "instr_address" : 'TCPIP0::192.168.0.105::inst0::INSTR',
    # "instr_address" : 'TCPIP0::K-N5231B-57006.local::inst0::INSTR',
}

PNA_X = VNA_Keysight(VNA_Keysight_InstrConfig, debug=True)

